### I Подключение библиотек

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

# Подключаем необходимые библиотеки для машинного обучения
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

### II Загружаем данные

In [4]:
item = 'nassCDS' # Выбираем из поля Item нашего датасета
package = 'DAAG' # Выбираем из поля Package нашего датасета

dfnew = sm.datasets.get_rdataset(item, package , cache=True).data
dfnew.head()

,dvcat,weight,dead,airbag,seatbelt,frontal,sex,ageOFocc,yearacc,yearVeh,abcat,occRole,deploy,injSeverity,caseid
0,25-39,25.069,alive,none,belted,1,f,26,1997,1990.0,unavail,driver,0,3.0,2:3:1
1,10-24,25.069,alive,airbag,belted,1,f,72,1997,1995.0,deploy,driver,1,1.0,2:3:2
2,10-24,32.379,alive,none,none,1,f,69,1997,1988.0,unavail,driver,0,4.0,2:5:1
3,25-39,495.444,alive,airbag,belted,1,f,53,1997,1995.0,deploy,driver,1,1.0,2:10:1
4,25-39,25.069,alive,none,belted,1,f,32,1997,1988.0,unavail,driver,0,3.0,2:11:1


### III Проведем очистку данных от пропусков, выбросов и некорректных значений¶

In [6]:
# Проверяем, есть ли в датасете дубликаты строк

dfnew[dfnew.duplicated()]     # Дубликатов нет

,dvcat,weight,dead,airbag,seatbelt,frontal,sex,ageOFocc,yearacc,yearVeh,abcat,occRole,deploy,injSeverity,caseid


In [7]:
# Поссмотрим пропуски
dfnew.isna().sum()

dvcat            0
weight           0
dead             0
airbag           0
seatbelt         0
frontal          0
sex              0
ageOFocc         0
yearacc          0
yearVeh          1
abcat            0
occRole          0
deploy           0
injSeverity    153
caseid           0
dtype: int64

In [8]:
# Убираем строки с пропусками
dfnew1 = dfnew[(dfnew['yearVeh']>1)&(dfnew['injSeverity']>=0)]
dfnew1.shape

(26063, 15)

In [9]:
# Выбросы будем искать во Возрасту человека, так как это единственное поле с множеством значений
# Возьмем метод через IQR

q1 = dfnew1['ageOFocc'].quantile(0.25)
q3 = dfnew1['ageOFocc'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5*iqr
upper_bound = q3 + 1.5*iqr
print(f"Выбросами будут считаться все возрасты, значения которых находится за пределами интервала [{lower_bound}, {upper_bound}]")

Выбросами будут считаться все возрасты, значения которых находится за пределами интервала [-17.0, 87.0]


In [10]:
# Формируем новый датасет без выбросов и пропусков
dfnew2 = dfnew1[((dfnew1['ageOFocc'] >= lower_bound) & (dfnew1['ageOFocc'] <= upper_bound)) | (dfnew1['ageOFocc'].isna())]

##### Возьмем датасет с фильтром по ДТП только при ФРОНТАЛЬНОМ УДАРЕ по ВОДИТЕЛЯМ

In [12]:
# Сформируем датасет с ДТП только при фронтальном ударе
df = dfnew2[(dfnew2['frontal'] == 1)&(dfnew2['occRole'] == 'driver')]
df.shape

(13353, 15)

### IV Итоговый датафрейм для работы

In [14]:
df.shape

(13353, 15)

In [15]:
df

,dvcat,weight,dead,airbag,seatbelt,frontal,sex,ageOFocc,yearacc,yearVeh,abcat,occRole,deploy,injSeverity,caseid
0,25-39,25.069,alive,none,belted,1,f,26,1997,1990.0,unavail,driver,0,3.0,2:3:1
1,10-24,25.069,alive,airbag,belted,1,f,72,1997,1995.0,deploy,driver,1,1.0,2:3:2
2,10-24,32.379,alive,none,none,1,f,69,1997,1988.0,unavail,driver,0,4.0,2:5:1
3,25-39,495.444,alive,airbag,belted,1,f,53,1997,1995.0,deploy,driver,1,1.0,2:10:1
4,25-39,25.069,alive,none,belted,1,f,32,1997,1988.0,unavail,driver,0,3.0,2:11:1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26209,10-24,261.217,alive,airbag,belted,1,m,76,2002,1991.0,deploy,driver,1,0.0,82:106:1
26212,25-39,3179.688,alive,none,belted,1,m,17,2002,1985.0,unavail,driver,0,0.0,82:107:1
26213,10-24,71.228,alive,airbag,belted,1,m,54,2002,2002.0,nodeploy,driver,0,2.0,82:108:2
26214,10-24,10.474,alive,airbag,belted,1,f,27,2002,1990.0,deploy,driver,1,3.0,82:110:1


### IV На основе проведенных наблюдений о взаимосвязи между переменными сформулируем задачу классификации или регрессии, которая может иметь практическую ценность

##### Минздрав нас всегда предупреждает... А что, если нас будет предупреждать ИИ в автомобилях. Предположим: недалекое будущее, в каждом автомобиле имеется ИИ для контроля и управления автомобилем. Туда забиваются все данные о водителе и автомобиле, а также текущей информации в процессе поездки. На приборной панели на экране есть значки предупреждения, и одно из них: СМЕРТЬ!.. красным шрифтом. Данный знак будет выходить при наличии некоторых параметров (данные о машине и водителе), при которых ИИ будет понимать, что водителю при аварии грозит смерть. ИИ также будет сигнализировать и мероприятиях, уменьшающих данные риски (уменьшающие вероятность смерти при ДТП). Соответственно водитель, увидев такой сигнал, должен принять соответствующие меры: уменьшить скорость, пристегнуть ремень, или вообще в силу своего возраста, пола и комплектации автомобиля - купить новый укомплектованный различными защитами автомобиль.

In [18]:
# За столбец таргет возьмем столбец Смертьность
df['target'] = np.where(df['dead'] == 'dead', 1, 0)
df.head()

C:\Users\RusKV\AppData\Local\Temp\ipykernel_13216\3197796964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = np.where(df['dead'] == 'dead', 1, 0)


,dvcat,weight,dead,airbag,seatbelt,frontal,sex,ageOFocc,yearacc,yearVeh,abcat,occRole,deploy,injSeverity,caseid,target
0,25-39,25.069,alive,none,belted,1,f,26,1997,1990.0,unavail,driver,0,3.0,2:3:1,0
1,10-24,25.069,alive,airbag,belted,1,f,72,1997,1995.0,deploy,driver,1,1.0,2:3:2,0
2,10-24,32.379,alive,none,none,1,f,69,1997,1988.0,unavail,driver,0,4.0,2:5:1,0
3,25-39,495.444,alive,airbag,belted,1,f,53,1997,1995.0,deploy,driver,1,1.0,2:10:1,0
4,25-39,25.069,alive,none,belted,1,f,32,1997,1988.0,unavail,driver,0,3.0,2:11:1,0


### V Подготовим переменных

##### Столбцы Вес (weight), год аварии (yearacc), Наличие подушки и срабатывание (abcat), факт срабатывания подушки (deploy), серъезность аварии (injSeverity), ИД (caseid) использовать не будем по причине их нелогичности и ненужности в нашей сформулированной задаче

In [21]:
# Категориальные переменные
#cat_vars = ['dvcat', 'airbag', 'seatbelt', 'sex']
cat_vars = ['dvcat', 'airbag', 'seatbelt', 'sex']
# Количественные переменные
#num_vars = ['ageOFocc', 'yearVeh']
num_vars = ['ageOFocc', 'yearVeh']

target_var = ['dead']

In [22]:
df[cat_vars] = df[cat_vars].astype(str)

C:\Users\RusKV\AppData\Local\Temp\ipykernel_13216\2577052954.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cat_vars] = df[cat_vars].astype(str)


#### Переведем категориальные переменные в числовой формат

In [24]:
# Закодируем категориальные переменные числами
X = pd.get_dummies(df[cat_vars], drop_first = True, dtype = int)
X.head()

,dvcat_10-24,dvcat_25-39,dvcat_40-54,dvcat_55+,airbag_none,seatbelt_none,sex_m
0,0,1,0,0,1,0,0
1,1,0,0,0,0,0,0
2,1,0,0,0,1,1,0
3,0,1,0,0,0,0,0
4,0,1,0,0,1,0,0


In [100]:
# Закодируем категориальные переменные числами
y = pd.get_dummies(df[target_var], drop_first = True, dtype = int)
y.head()

NameError: name 'target_var' is not defined

#### Нормализуем количественные переменные (в шкалу от 0 до 1)

In [26]:
scaler = MinMaxScaler()
scaler.fit(df[num_vars])

MinMaxScaler()

In [27]:
X[num_vars] = scaler.transform(df[num_vars])
X.head()

,dvcat_10-24,dvcat_25-39,dvcat_40-54,dvcat_55+,airbag_none,seatbelt_none,sex_m,yearVeh
0,0,1,0,0,1,0,0,0.74
1,1,0,0,0,0,0,0,0.84
2,1,0,0,0,1,1,0,0.70
3,0,1,0,0,0,0,0,0.84
4,0,1,0,0,1,0,0,0.70


#### Теперь переменные готовы для обучения!!!

### VI Разделим датасет на обучающую (60%) и тестовую (40%) выборки. Далее тестовую выборку разделим пополам (20% и 20%)

##### 1) Первую половину будем использовать для оценки качества модели. Предполагаем, что здесь мы знаем наблюдаемые значения таргета, и можем сравнивать их с предсказаниями

##### 2) Вторую половину будем использовать для построения прогнозов. Предполагаем, что здесь значения таргета нам неизвестны – как в практической ситуации использования моделей прогнозирования.

In [32]:
y = df['target'].values
# Разделяем переменные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

# Разделим тестовую выборку пополам
X_test1, X_test2, y_test1, y_test2 = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

### VII Обучим различные модели, проведем оценку их качества на первой половине тестовой выборки

#### 1) Модель KNeighborsClassifier

In [35]:
# Обучаем модель на обучающей части данных с помощью модели классификации KNN
neigh = KNeighborsClassifier(n_neighbors = 3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [36]:
# Предсказываем для тестовой части значение целевой переменной
y_pred = neigh.predict(X_test1)

In [37]:
# Количество реальных смертельных исходов
np.sum(y_test1)

85

In [38]:
# Количество предсказанных смертельных исходов
np.sum(y_pred)

31

In [39]:
# Сравниваем предсказанные значения с реальными значениями
pd.DataFrame(confusion_matrix(y_test1, y_pred), index = ['True 0', 'True 1'], columns = ['Pred 0', 'Pred 1'])

,Pred 0,Pred 1
True 0,2557,29
True 1,83,2


##### Из 85 реальных смертельных исходов, предсказаны только 6. Из 40 предсказанных смертельных исходов, реально было 6 смертелььных случаев.

In [41]:
# Рассмотрим метрики качества модели
print('Precision', precision_score(y_test1, y_pred))
print('Recall', recall_score(y_test1, y_pred))

Precision 0.06451612903225806
Recall 0.023529411764705882


##### Качество модели - очень низкое. Процент смертей, которые модель корректно обозначает, составялет 15%, а процент реальных смертей, которых модель способна поймать - 7%.

##### Пробуем подобрать значение гиперпараметра

In [44]:
for i in range(3, 30, 2):
    neigh = KNeighborsClassifier(n_neighbors = i)
    neigh.fit(X_train, y_train)
    y_pred = neigh.predict(X_test1)
    print('Neigbors:', i, 
          'Precision:', np.round(precision_score(y_test1, y_pred),3), 
          'Recall', np.round(recall_score(y_test1, y_pred),3)
         )

Neigbors: 3 Precision: 0.065 Recall 0.024
Neigbors: 5 Precision: 0.0 Recall 0.0
Neigbors: 7 Precision: 0.0 Recall 0.0
Neigbors: 9 Precision: 0.0 Recall 0.0
Neigbors: 11 Precision: 0.0 Recall 0.0
Neigbors: 13 Precision: 0.0 Recall 0.0
Neigbors: 15 Precision: 0.0 Recall 0.0
Neigbors: 17 Precision: 0.0 Recall 0.0
Neigbors: 19 Precision: 0.0 Recall 0.0
Neigbors: 21 Precision: 0.0 Recall 0.0


C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Neigbors: 23 Precision: 0.0 Recall 0.0


C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Neigbors: 25 Precision: 0.0 Recall 0.0


C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Neigbors: 27 Precision: 0.0 Recall 0.0
Neigbors: 29 Precision: 0.0 Recall 0.0


C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##### При значении параметра 9 - получаются оптимальные значения: процент смертей, которые модель корректно обозначает, составялет 25%, а процент реальных смертей, которых модель способна поймать - 5,9%.
##### При значении параметра 19 - процент смертей, которые модель корректно обозначает, составялет 50%, а процент реальных смертей, которых модель способна поймать - всего 1,2%.

#### 2) Модель SVM

In [47]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(X_train, y_train)
y_pred = svc_clf.predict(X_test1)
print(
    'Precision:', np.round(precision_score(y_test1, y_pred),3), 
    'Recall', np.round(recall_score(y_test1, y_pred),3)
)

Precision: 0.0 Recall 0.0


C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##### Модель SVM ни одну смерть не видит и не обозначает корректно.

#### 3) Модель Decision tree

In [50]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)
y_pred = tree_clf.predict(X_test1)
print(
    'Precision:', np.round(precision_score(y_test1, y_pred),3), 
    'Recall', np.round(recall_score(y_test1, y_pred),3)
)

Precision: 0.0 Recall 0.0


##### В модели Decision tree - процент смертей, которые модель корректно обозначает, составялет 14,1%, а процент реальных смертей, которых модель способна поймать - 15,3%.

#### 4) Модель Benchmark - модель случайного предсказания

In [53]:
prob = np.mean(y_test1)
from scipy.stats import bernoulli
y_pred = bernoulli.rvs(prob, size = len(y_test1))
print(
    'Precision:', np.round(precision_score(y_test1, y_pred),3), 
    'Recall', np.round(recall_score(y_test1, y_pred),3)
)

Precision: 0.0 Recall 0.0


##### В модели Benchmark - процент смертей, которые модель корректно обозначает, составялет 2.4%, а процент реальных смертей, которых модель способна поймать - 2.4%.

#### 4) Модель Random Forest

In [56]:
from sklearn.ensemble import RandomForestClassifier
for i in range(1, 30, 2):
    rf_clf = RandomForestClassifier(max_depth=i, random_state=42)
    rf_clf.fit(X_train, y_train)
    y_pred = rf_clf.predict(X_test)
    print('Depth:', i, 
          'Precision:', np.round(precision_score(y_test, y_pred),3), 
          'Recall', np.round(recall_score(y_test, y_pred),3)
         )

C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Depth: 1 Precision: 0.0 Recall 0.0


C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Depth: 3 Precision: 0.0 Recall 0.0


C:\Users\RusKV\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Depth: 5 Precision: 0.0 Recall 0.0
Depth: 7 Precision: 0.5 Recall 0.005
Depth: 9 Precision: 0.3 Recall 0.032
Depth: 11 Precision: 0.259 Recall 0.037
Depth: 13 Precision: 0.175 Recall 0.037
Depth: 15 Precision: 0.175 Recall 0.037
Depth: 17 Precision: 0.175 Recall 0.037
Depth: 19 Precision: 0.175 Recall 0.037
Depth: 21 Precision: 0.175 Recall 0.037
Depth: 23 Precision: 0.175 Recall 0.037
Depth: 25 Precision: 0.175 Recall 0.037
Depth: 27 Precision: 0.175 Recall 0.037
Depth: 29 Precision: 0.175 Recall 0.037


##### При значении параметра 5 - процент смертей, которые модель корректно обозначает, составялет 66,7%, а процент реальных смертей, которых модель способна поймать - 2,1%.
##### При значении параметра 23 - получаются оптимальные значения: процент смертей, которые модель корректно обозначает, составялет 23,8%, а процент реальных смертей, которых модель способна поймать - всего 12,8%.

### VIII Выберем модель и набор переменных, которые дают наиболее удовлетворительное качество..

#### В нашем анализе важна метрика Recall - это процент реальных смертей, которые модель способна поймать (чем выше это число, тем лучше), тем боллее вероятно, что человеку на экране будет высвечиваться реальные предсказания-предупреждения. И нам практически не важна метрика Precision, так как если ИИ спрогнозирует фейковую смерть на экране, и человек произведет мероприятия по уменьшению вероятности смерти, то в этом нет ничего плохого, наоборот даже лучше, так сказать с запасом))).

1) Из расчетов, проделанных выше, метрика Recall выше у модели Decision tree - 15,3%.

2. Проделал аналогичные расчеты в другом файлике, убрав одну количествунную переменную "yearVeh" (год выпуска авто). Метрика Recall выше у модели Decision tree - 12,9%.

3. Проделал аналогичные расчеты в другом файлике, переместив переменную "yearVeh" (год выпуска авто) из блока количественных и блок категориальных. Метрика Recall выше у модели Decision tree - 16,5%.